In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pygame
import random
from pygame import mixer
import time as t
cap=cv2.VideoCapture(0)
mp_hands=mp.solutions.hands
mp_draw=mp.solutions.drawing_utils
Hands=mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5,max_num_hands=4)
pygame.init()
mixer.init()
mixer.music.load('Sounds/Komiku_-_12_-_Bicycle.mp3')
mixer.music.set_volume(0.5)
mixer.music.play(-1)
def get_hand(frame):
    is_closed=False
    h,w,c=frame.shape
    frame_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame_rgb.flags.writeable=False
    results=Hands.process(frame_rgb)
    frame_rgb.flags.writeable=True
    if results.multi_hand_landmarks:
        for land_marks in results.multi_hand_landmarks:
            #pass
            mp_draw.draw_landmarks(frame,land_marks,mp_hands.HAND_CONNECTIONS)
            X=land_marks.landmark[0].x 
            Y=land_marks.landmark[0].y 
            Z=land_marks.landmark[0].z 
            main=(int(X*w),int(Y*h))
            thumb=(int((land_marks.landmark[4].x)*w) ,int((land_marks.landmark[4].y)*h))
            pointer=(int((land_marks.landmark[8].x)*w) ,int((land_marks.landmark[8].y)*h))
            middle=(int((land_marks.landmark[12].x)*w) ,int((land_marks.landmark[12].y)*h))
            ring =(int((land_marks.landmark[16].x)*w) ,int((land_marks.landmark[16].y)*h))
            index=(int((land_marks.landmark[20].x)*w) ,int((land_marks.landmark[20].y)*h))

            cv2.circle(frame,main,5,(255,0,0),-1)
            
            cv2.circle(frame,thumb,5,(255,0,0),-1)
            cv2.circle(frame,pointer,5,(255,0,0),-1)
            cv2.circle(frame,middle,5,(255,0,0),-1)
            cv2.circle(frame,ring,5,(255,0,0),-1)
            cv2.circle(frame,index,5,(255,0,0),-1)
            hull=cv2.convexHull(np.array([thumb,pointer,middle,ring,index,main]))
            
            pp=len(hull)
            if pp<=5:
                is_closed=True
            palm=np.array([(int((land_marks.landmark[5].x)*w) ,int((land_marks.landmark[5].y)*h)),
                       (int((land_marks.landmark[9].x)*w) ,int((land_marks.landmark[9].y)*h)),
                       (int((land_marks.landmark[13].x)*w) ,int((land_marks.landmark[13].y)*h)),
                       (int((land_marks.landmark[17].x)*w) ,int((land_marks.landmark[17].y)*h)),main])
            try:
                M = cv2.moments(palm)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                cv2.circle(frame,(cX,cY),5,(255,0,0),-1)
            except:
                pass
            #cv2.putText(frame,str(pp),(20,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
            cv2.polylines(frame,[hull],True,(0,0,255),2)
            return is_closed,(cX,cY)
        
class Game:
    def __init__(self):
        self.run=True
        self.is_closed=False
        self.width=800
        self.height=650
        self.hand_width,self.hand_height=95,95
        self.fps=60
        self.velocity=4
        self.size=40
        self.score=0
        self.duration=1
        self.white=(255,255,255)
        self.black=(0,0,0)
        self.curr_time=0
        
        self.directions=['r','l']
        self.mosquitos=[]
        self.dirs=[]
        self.mosquitos_to_be_removed=[]
        self.mosquito_event=pygame.USEREVENT+1
        self.collid=pygame.USEREVENT+2
        
        self.collid_sound=mixer.Sound('Sounds/slap.wav')
        self.font = pygame.font.SysFont('comicsans', 40)
        
        self.collid_sound.set_volume(1.0)
        self.clock=pygame.time.Clock()
        pygame.display.set_caption('mosquito killer')
        pygame.time.set_timer(self.mosquito_event,1000)
        
        self.mosquito=pygame.image.load('mosquito/mosquito.png')
        self.hand=pygame.transform.scale(pygame.image.load('hand.png'),(self.hand_width,self.hand_height))
        self.closed=pygame.transform.scale(pygame.image.load('closed.png'),(self.hand_width,self.hand_height))
        self.background=pygame.transform.scale(pygame.image.load('background.jpg'),(self.width,self.height))
        self.win=pygame.display.set_mode((self.width,self.height))
        
        self.init_time=t.time()

    def draw_mosq(self):
        self.mosquitos_to_be_removed=[]
        
        for i, mosq in enumerate(self.mosquitos):
            
            if self.dirs[i]=='l':
                self.win.blit(pygame.transform.scale(self.mosquito,(self.size,self.size)),(mosq.x,mosq.y))
                self.mosquitos[i].x-=self.velocity
            if self.dirs[i]=='r':
                
                self.win.blit(pygame.transform.flip(pygame.transform.scale(self.mosquito,(self.size,self.size)),True,False),(mosq.x,mosq.y))
                self.mosquitos[i].x+=self.velocity
            if mosq.x >self.width+50 or mosq.x<-50:
                self.mosquitos_to_be_removed.append(i)
            try:
                if mosq.colliderect(self.hand_rect):
                    if self.is_closed:
                        self.mosquitos_to_be_removed.append(i)
                        pygame.event.post(pygame.event.Event(self.collid))
                    #print('l')
            except:
                pass
        
        for j in self.mosquitos_to_be_removed:
            try:
                self.mosquitos.pop(j)
            except:
                pass
        
    def main(self):
        try:
            while self.run:

                self.clock.tick(self.fps)
                for event in pygame.event.get():
                    if event.type==pygame.QUIT:
                        self.run=False
                        pygame.quit()
                        cv2.destroyAllWindows()
                        cap.release()
                        return
                    if event.type==self.mosquito_event:
                        #print('pppp')
                        self.direction=self.directions[random.randint(0,1)]
                        if self.direction=='l':
                            self.mosquitos.append(pygame.Rect(random.randint(20,self.width-40),random.randint(50,self.height-40),self.size,self.size))
                            self.dirs.append(self.direction)
                        if self.direction=='r':
                            self.mosquitos.append(pygame.Rect(random.randint(20,self.width-40),random.randint(50,self.height-40),self.size,self.size))
                            self.dirs.append(self.direction)

                    if event.type==self.collid:
                        self.score+=1
                        self.collid_sound.play()

                _,frame=cap.read()
                frame=cv2.resize(frame,(self.width,self.height))
                frame=cv2.flip(frame,1,0)
                my_ret=get_hand(frame)
                cv2.imshow('frame',frame)

                ### drawing
                self.win.fill(self.white)
                self.win.blit(self.background,(0,0))
                score_font=self.font.render(f'Score: {self.score}',1,self.black)
                self.curr_time=t.time()

                remain=self.curr_time-self.init_time
                if 60-int(remain)==0:
                    self.duration=self.duration-1
                    self.init_time=self.curr_time
                time_font=self.font.render(f'{self.duration-1} : {60-int(remain)}',1,self.black)
                if(self.duration==0 and 60-int(remain)==0 ):
                    game_over=self.font.render(f'Game Over Your Score is :{self.score}',1,self.black)
                    self.win.blit(game_over,(self.width//2-game_over.get_width()//2,self.height//2-game_over.get_height()//2))
                    pygame.display.update()
                    pygame.time.delay(5000)
                    self.run=False
                    pygame.quit()
                    cv2.destroyAllWindows()
                    cap.release()
                    return


                self.win.blit(score_font,(10,10))
                self.win.blit(time_font,(self.width-time_font.get_width()-10,10))
                if my_ret is not None:
                    self.is_closed,(cx,cy)=my_ret

                    #print((cx,cy))
                    self.hand_rect=pygame.Rect(cx,cy,self.hand_width,self.hand_height)
                    if self.is_closed is False:
                        self.win.blit(self.hand,(self.hand_rect.x,self.hand_rect.y))
                    if self.is_closed is True:
                        self.win.blit(self.closed,(self.hand_rect.x,self.hand_rect.y))                    
                    #pygame.draw.rect(self.win,(255,0,0),self.hand_rect)
                self.draw_mosq()
                pygame.display.update()
        except:
            pass
        
if __name__=='__main__':
    mygame=Game()
    mygame.main()
            
        

pygame 2.0.1 (SDL 2.0.14, Python 3.8.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


5